In [4]:
# 1、灰度化（将图像看做一个x,y,z（灰度）的三维图像）；
# 2、 采用Gamma校正法对输入图像进行颜色空间的标准化（归一化）；；
# 3、 计算图像每个像素的梯度（包括大小和方向）；
# 4、 将图像划分成小cells；
# 5、 统计每个cell的梯度直方图（不同梯度的个数），得到cell的描述子；
# 6、 将每几个cell组成一个block，得到block的描述子；
# 7、 将图像image内的所有block的HOG特征descriptor串联起来就可以得到HOG特征，该特
# 8、征向量就是用来目标检测或分类的特征。

In [8]:
import cv2
import numpy as np

#判断矩形i是否包含矩形o
def is_inside(o,i):
    ox,oy,ow,oh=o
    ix,iy,iw,ih=i
    return ox > ix and oy > iy and ox + ow < ix + iw and oy + oh < iy + ih
#绘画人体颜色框
def draw_person(image,person):
    x,y,w,h=person
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,255),2)
img=cv2.imread("people.jpg")  
hog = cv2.HOGDescriptor()  # 启动检测器对象
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())  # 指定检测器类型为人体
found, w = hog.detectMultiScale(img,0.1,(2,2)) # 加载并检测图像
print(found)
print(w)
#丢弃冗余矩形
found_filtered = []
for ri, r in enumerate(found):
    for qi, q in enumerate(found):
        if ri != qi and is_inside(r, q):
            break
    else:
        found_filtered.append(r)
#对有效矩形进行框定
for person in found_filtered:
    draw_person(img, person)
cv2.imshow("people detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[322 123  92 184]
 [ 97 109  92 184]
 [228 106  94 188]
 [157 143  89 178]]
[1.65200859 2.16342618 1.38857961 1.30450152]


In [9]:
# 1、计算图像在X和Y方向的梯度；
# 2、计算图像两个方向梯度的乘积，IxIx,IyIy,IxIy；
# 3、使用高斯函数对三者进行高斯加权，生成矩阵M的A,B,C；
# 4、计算每个像素的Harris响应值R，并对小于某一阈值t的R置为零；
# 5、在3×3或5×5的邻域内进行非最大值抑制，局部最大值点即为图像中的角点；

In [19]:
import cv2
import numpy as np


filename = 'harris2.png'
img = cv2.imread(filename)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

# 输入图像必须是 float32 ,最后一个参数在 0.04 到 0.06 之间
dst = cv2.cornerHarris(gray,2,3,0.06)

#结果进行膨胀，可有可无
dst = cv2.dilate(dst,None)
print(dst)
# 设定阈值，不同图像阈值不同
img[dst>0.01*dst.max()]=[0,0,255]
print(dst.max())
cv2.imshow('dst',dst)
cv2.imshow('dst_img',img)
#cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
533091900.0


In [20]:
# 1、尺度空间极值检测点检测
# 2、关键点定位：去除一些不好的特征点，保存下来的特征点能够满足稳定性等条件
# 3、关键点方向参数：获取关键点所在尺度空间的邻域，然后计算该区域的梯度和方向，根据计算得到的结果创建方向直方图，直方图的峰值为主方向的参数
# 4、关键点描述符：每个关键点用一组向量（位置、尺度、方向）将这个关键点描述出来，使其不随着光照、视角等等影响而改变
# 5、关键点匹配：分别对模板图和实时图建立关键点描述符集合，通过对比关键点描述符来判断两个关键点是否相同

In [23]:
import cv2
import numpy as np


img=cv2.imread('harris2.png')
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
shift=cv2.SIFT_create()
kp=shift.detect(gray,None)
img=cv2.drawKeypoints(gray,kp,img)
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
import cv2
import numpy as np

def LBP(src):
    '''
    :param src:灰度图像
    :return:
    '''
    height = src.shape[0]
    width = src.shape[1]
    dst = src.copy()
    lbp_value = np.zeros((1,8), dtype=np.uint8)
    #print(lbp_value)
    neighbours = np.zeros((1,8), dtype=np.uint8)
    #print(neighbours)
    for x in range(1, width-1):
        for y in range(1, height-1):
            neighbours[0, 0] = src[y - 1, x - 1]
            neighbours[0, 1] = src[y - 1, x]
            neighbours[0, 2] = src[y - 1, x + 1]
            neighbours[0, 3] = src[y, x - 1]
            neighbours[0, 4] = src[y, x + 1]
            neighbours[0, 5] = src[y + 1, x - 1]
            neighbours[0, 6] = src[y + 1, x]
            neighbours[0, 7] = src[y + 1, x + 1]
            center = src[y, x]
            for i in range(8):
                if neighbours[0, i] > center:
                    lbp_value[0, i] = 1
                else:
                    lbp_value[0, i] = 0

            lbp = lbp_value[0, 0] * 1 + lbp_value[0, 1] * 2 + lbp_value[0, 2] * 4 + lbp_value[0, 3] * 8 \
                + lbp_value[0, 4] * 16 + lbp_value[0, 5] * 32 + lbp_value[0, 6] * 64 + lbp_value[0, 7] * 128
            
            #print(lbp)
            dst[y, x] = lbp

    return dst

img = cv2.imread('people.jpg',0)
print(img.shape)
cv2.imshow('src',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
new_img = LBP(img)

cv2.imshow('dst',new_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(334, 500)


In [25]:
#模板匹配
import cv2
import numpy as np
def template_demo(tpl,target):

    methods = [cv2.TM_SQDIFF_NORMED, cv2.TM_CCORR_NORMED, cv2.TM_CCOEFF_NORMED]   #3种模板匹配方法
    th, tw = tpl.shape[:2]
    for md in methods:
        #print(md)
        result = cv2.matchTemplate(target, tpl, md)
        #print(result.shape)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        print(min_val, max_val, min_loc, max_loc)
        if md == cv2.TM_SQDIFF_NORMED:
            tl = min_loc
        else:
            tl = max_loc
        br = (tl[0]+tw, tl[1]+th)   #br是矩形右下角的点的坐标
        cv2.rectangle(target, tl, br, (0, 0, 255), 2)
        cv2.namedWindow("match-" + np.str(md), cv2.WINDOW_NORMAL)
        cv2.imshow("match-" + np.str(md), target)
        
tpl =cv2.imread("sample2.jpg")
print(tpl.shape)
target = cv2.imread("target1.jpg")
print(target.shape)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.namedWindow('template image', cv2.WINDOW_NORMAL)
cv2.imshow("template image", tpl)
cv2.namedWindow('target image', cv2.WINDOW_NORMAL)
cv2.imshow("target image", target)
template_demo(tpl,target)
cv2.waitKey(0)
cv2.destroyAllWindows()

(136, 92, 3)
(643, 903, 3)
0.313273549079895 1.0 (164, 186) (61, 0)
0.39695343375205994 0.8127202391624451 (232, 111) (164, 186)
-0.3538476526737213 0.5110524296760559 (356, 64) (164, 186)


C:\Users\22967\AppData\Local\Temp/ipykernel_9212/4273129370.py:20: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv2.namedWindow("match-" + np.str(md), cv2.WINDOW_NORMAL)
C:\Users\22967\AppData\Local\Temp/ipykernel_9212/4273129370.py:21: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv2.imshow("match-" + np.str(md), target)


In [1]:
import cv2

# 读入图像
img = cv2.imread("3.png")

# 加载人脸特征，该文件在 python安装目录\Lib\site-packages\cv2\data 下
face_cascade = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
# 将读取的图像转为COLOR_BGR2GRAY，减少计算强度
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 检测出的人脸个数
faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.15, minNeighbors = 4, minSize = (5, 5))

print("Face : {0}".format(len(faces)))
print(faces)
# 用矩形圈出人脸的位置
for(x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2) 

cv2.namedWindow("Faces")
cv2.imshow("Faces", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Face : 1
[[ 68  35 249 249]]


In [1]:
# -*- coding：utf-8 -*-
import cv2
import dlib
import numpy as np


predictor_model = './shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_model)

# cv2读取图像
test_film_path = "./3.png"
img = cv2.imread(test_film_path)
# 取灰度
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# 人脸数rects
rects = detector(img_gray, 0)
print(rects[0])
for i in range(len(rects)):
    landmarks = np.matrix([[p.x, p.y] for p in predictor(img,rects[i]).parts()])
    print(landmarks, type(landmarks))
    for idx, point in enumerate(landmarks):
        # 68点的坐标
        pos = (point[0, 0], point[0, 1])
        #print(idx+1, pos)

        # 利用cv2.circle给每个特征点画一个圈，共68个
        cv2.circle(img, pos, 3, color=(0, 255, 0))
        # 利用cv2.putText输出1-68
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,str(idx+1),pos,font,0.5,(0, 0, 255),1,cv2.LINE_AA)
#         cv2.putText(img, str(idx+1), (123,456), font, 2, (0,255,0), 3)
#cv2.imwrite("result.png", img)
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

[(90, 66) (305, 281)]
[[ 95 137]
 [ 98 162]
 [104 187]
 [108 212]
 [117 235]
 [131 255]
 [149 272]
 [170 285]
 [192 289]
 [215 285]
 [235 270]
 [253 251]
 [266 229]
 [272 205]
 [276 180]
 [283 156]
 [285 131]
 [101 114]
 [116 106]
 [134 107]
 [153 110]
 [172 117]
 [209 117]
 [227 109]
 [246 106]
 [265 105]
 [282 113]
 [190 135]
 [191 153]
 [191 172]
 [192 189]
 [177 199]
 [184 203]
 [192 205]
 [201 203]
 [208 199]
 [125 135]
 [136 127]
 [151 127]
 [163 139]
 [149 143]
 [135 143]
 [219 138]
 [230 127]
 [245 126]
 [257 134]
 [247 142]
 [232 142]
 [161 235]
 [172 226]
 [185 219]
 [193 222]
 [200 219]
 [212 226]
 [224 235]
 [213 247]
 [201 252]
 [193 252]
 [184 252]
 [172 247]
 [168 235]
 [185 231]
 [193 232]
 [200 232]
 [217 235]
 [201 235]
 [193 236]
 [185 235]] <class 'numpy.matrix'>
